In [ ]:
from splashback.cluster import cluster_sample
from astropy.table import Table
import numpy as np
from astropy import units as u
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import ScalarFormatter, LogLocator
from splashback.profile import NFW, Mvir_to_M200m
import emcee
from splashback.profile import DK14, rho, rho_infall, rho_Ein, f_trans
from scipy import stats
plt.rc('font', size=20)

***Cluster Table***

In [ ]:
data = Table.read('data/CCCPnomergers.fits')
data.sort('m_g')

print "Rows:"

for i in xrange(len(data)):
        print data['name'][i], "& - & - &", \
        np.around(data['z'][i], 3), "&", \
        np.around(data['beta'][i], 3), "&", \
        format(data['m_g'][i]/1e13, '.1f'), "&", \
        format(data['m_200'][i]/1e14, '.1f'), "\\\\"

***Figure 1***

In [ ]:
#STACK
z_pivot = 0.25
bin_edges = np.geomspace(.2, 9, 11)*u.Mpc


CCCP = cluster_sample('data/CCCPnomergers.fits', 'data/source/')
CCCP.stack_ESD(bin_edges, cosmic_noise='output/lss_covariance/', comoving=True, mscaling=False, weighted=True, raw=True)

g_PSF = CCCP.ESD.value

CCCP = cluster_sample('data/CCCPnomergers.fits', 'data/source/')
CCCP.stack_ESD(bin_edges, cosmic_noise='output/lss_covariance/', comoving=True, mscaling=False, weighted=True, contamination=False)

g_contamination = CCCP.ESD.value

CCCP = cluster_sample('data/CCCPnomergers.fits', 'data/source/')
CCCP.stack_ESD(bin_edges, cosmic_noise='output/lss_covariance/', comoving=True, mscaling=False, weighted=True)
g_full = CCCP.ESD.value
g_full_err = CCCP.ESDerr

y = CCCP.ESD.to('Msun/Mpc/Mpc').value
Cov = CCCP.Cov
Covinv = np.linalg.inv(Cov[:,:])

In [ ]:
#FIT

Mvirarray, carray = np.meshgrid(np.linspace(12., 18., 300), np.geomspace(3., 6., 100))
Mvirarray = Mvirarray.flatten()
carray = carray.flatten()

L = np.zeros(len(Mvirarray))
for i in xrange(len(Mvirarray)):
    Mvir = Mvirarray[i]*1e14*u.Msun
    c= carray[i]
    z = z_pivot
    
    prof = NFW(Mvir, z, c)
    model = prof.D_Sigma_reduced(CCCP.rbin.to('Mpc').value/(1.+z_pivot), CCCP.Sigma_crit.mean().to('Msun/Mpc/Mpc').value)
    L[i] = (-0.5*np.matrix(y[:5]-model[:5])*Covinv*np.matrix(y[:5]-model[:5]).T).sum()

print "Mvir, z, c: ", Mvirarray[L.argmax()], z_pivot, carray[L.argmax()]#, Mvirarray[L.argmax()]
M200plot, r200plot =  Mvir_to_M200m(Mvirarray[L.argmax()]*1e14*u.Msun, z=z_pivot, also_r=True, c=carray[L.argmax()])
print "M200m, r200m stack at: ", Mvir_to_M200m(Mvirarray[L.argmax()]*1e14*u.Msun, z=z_pivot, also_r=True)
print L.max()



In [ ]:
plt.figure(figsize=(8, 10))
gs = gridspec.GridSpec(4, 1)
ax1 = plt.subplot(gs[0:3, 0])
ax2 = plt.subplot(gs[3, 0])

y = CCCP.ESD.value
yerr = CCCP.ESDerr.value
ax1.errorbar(CCCP.rbin.value, y/1e14, yerr/1e14, c='r', capsize=5, fmt='o')
y = CCCP.ESD.value
yerr = CCCP.ESD_stat_err.value
ax1.errorbar(CCCP.rbin.value, y/1e14, yerr/1e14, c='r', capsize=5, fmt='o', label='CCCP all')

prof = NFW(Mvirarray[L.argmax()]*1e14*u.Msun, z_pivot, c=carray[L.argmax()])
model = prof.D_Sigma_reduced(CCCP.rbin.to('Mpc').value/(1.+z_pivot), CCCP.Sigma_crit.mean().to('Msun/Mpc/Mpc').value)

ax1.plot(CCCP.rbin.value, model/1e14, label='NFW best-fit', c='k')
ax1.arrow(r200plot.value*(1.+z_pivot), 0.12, 0, -0.02, lw=1, width=0.05, head_length=0.01, fc='k')


ax1.set_yscale('log')
ax1.tick_params(labelbottom=False)  
ax1.tick_params(top=True, which='both', direction='inout')
ax1.tick_params(axis='y', which='minor')
ax1.tick_params(length=10, which='major')
ax1.tick_params(length=5, which='minor')
ax1.set_yticks([0.5,1, 2, 3])
ax1.yaxis.set_major_formatter(ScalarFormatter())
ax1.legend(frameon=False)
ax1.set_ylabel("$\\langle g_t \\Sigma_\\mathrm{cr} \\rangle$ ($10^{14}$ M$_\\odot$ Mpc$^{-2}$)", multialignment='center')


y = (g_full-g_PSF)/g_full_err
ax2.plot(CCCP.rbin.value, y, label='PSF', c='k', ls=":")
y = 0.02*g_full/g_full_err
ax2.plot(CCCP.rbin.value, y, label='cluster member\n contamination', c='k', ls='--', lw=2)


ax2.tick_params(top=True, which='both', direction='inout')
ax2.tick_params(length=10, which='major')
ax2.tick_params(length=5, which='minor')
ax1.set_xscale('log')
ax2.set_xscale('log')
ax1.set_xlim([.19, 10])
ax2.set_xlim([.19, 10])
ax2.set_ylim([-0.08, 1.])
ax2.set_ylabel("$\\langle g_t \\Sigma_\\mathrm{cr} \\rangle$ ($10^{14}$ M$_\\odot$ Mpc$^{-2}$)", multialignment='center')
ax2.set_xlabel('$R$ (Mpc)')
ax2.set_ylabel(r"$\frac{\delta \langle g_t \Sigma_\mathrm{cr} \rangle }{\sqrt{\mathrm{Var }\langle g_t \Sigma_\mathrm{cr}\rangle}}$", size=28)
ax2.set_xticks([0.5,1, 2, 4, 8])
ax2.set_yticks([0.0,0.2, 0.4, 0.6, 0.8])
ax2.xaxis.set_major_formatter(ScalarFormatter())


plt.legend(fontsize=13, frameon=False, loc=1, bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.savefig("output/figures/1.pdf")




***Figure 3***

In [ ]:
bin_edges = np.geomspace(.2, 9, 11)*u.Mpc

plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(1, 4)
ax1 = plt.subplot(gs[0, :-2])
ax2 = plt.subplot(gs[0, 2:])


n_disc = 1000
n_samples = 20000
n_samples2 = 100

for chain_name, catalog_name, color, factor, factor2, label_plot in zip(['nomergers_True', 'highm_True'], ['nomergers', 'highm'], ['r', '#4286f4'], [1, 1.1], [1, 0.95], ['CCCP all', 'CCCP high mass']):
    
    # Load data
    CCCP = cluster_sample('data/CCCP'+catalog_name+'.fits', 'data/source/')
    CCCP.stack_ESD(bin_edges, cosmic_noise='output/lss_covariance/', comoving=True, mscaling=False, weighted=True)
    
    # Load fit
    filename = "output/chains/"+chain_name+'.h5'
    backend = emcee.backends.HDFBackend(filename, read_only=True)
    data = backend.get_chain(flat = True, discard=n_disc)
    
    
    #Temp variables
    params = data[np.random.choice(data.shape[0], n_samples), :]
    params2 = data[np.random.choice(data.shape[0], n_samples2), :]
    r = np.linspace(.2, 20., 1000)
    r2 = np.geomspace(.2, 10, 20)
    
    logr = np.log(r)
    r_sp = np.zeros(n_samples)
    der = np.zeros(n_samples)
    der_plot = np.zeros((n_samples, len(r)))
    der_collapse = np.zeros(n_samples)
    DSigma = np.zeros((n_samples2, len(r2)))

    
    # Contours for model, derivative
    for i in xrange(n_samples):
        rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e = params[i, :]

        #Total derivative 
        para_tot = [rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e]
        y_tot = rho(r, para_tot)
        logy_tot = np.log(y_tot)
        derivative = np.gradient(logy_tot, logr)
        der_plot[i] = derivative
        
        r_sp[i] = r[derivative.argmin()]
        der[i] = derivative.min()
        idx_rsp = derivative.argmin()

        #Collapse derivative
        #log_collapse = np.log(rho_Ein(r, [rho_s, r_s, logalpha])*f_trans(r, [r_t, logbeta, loggamma]))
        #derivative = np.gradient(log_collapse, logr)
        #der[i] = derivative[idx_rsp]

    for i in xrange(n_samples2):
        print i
        DSigma[i] = DK14(r2, params2[i, :])
        
        
    der_plot_up = np.percentile(der_plot, 100-15.86, axis=0)
    der_plot_down = np.percentile(der_plot, 15.86, axis=0)
    
    
    r_sp_up = np.percentile(r_sp, 100-15.86)
    r_sp_down = np.percentile(r_sp, 15.86)
    r_sp_med = np.percentile(r_sp, 50)
    print "r_sp:", r_sp_med, "+", r_sp_up-r_sp_med, "-", r_sp_med-r_sp_down
    
    der_up = np.percentile(der, 100-15.86)
    der_down = np.percentile(der, 15.86)
    der_up_up = np.percentile(der, 100-0.135)
    der_down_down = np.percentile(der, 0.135)
    der_med = np.percentile(der, 50)
    #print der_med, der_up, der_down
    print "der:", der_med, "+", der_up-der_med, "-", der_med-der_down
    print "der:", der_med, "+", der_up_up-der_med, "-", der_med-der_down_down


    data = backend.get_chain(flat = True, discard=n_disc)
    lnprob = backend.get_log_prob(flat=True, discard=n_disc)
    params_best = data[lnprob.argmax(), :]
    model_best = DK14(CCCP.rbin.value, params_best)
    ax1.errorbar(CCCP.rbin.value, CCCP.ESD.value/1e14, yerr=CCCP.ESDerr.value/1e14, color=color, fmt='o', label=label_plot)
    
    DSigma_plot_up = np.percentile(DSigma, 100-15.86, axis=0)
    DSigma_plot_down = np.percentile(DSigma, 15.86, axis=0)
    ax1.fill_between(r2, DSigma_plot_down*10, DSigma_plot_up*10, color=color, alpha=0.3)
    
    
    ax2.plot(r, der_plot_up, c=color, lw=2)
    ax2.plot(r, der_plot_down, c=color, lw=2)
    ax2.fill_between(r, der_plot_down, der_plot_up, color=color, alpha=0.3, label=label_plot+" fit")
    #ax2.errorbar([r_sp_med], [-4*factor2], xerr=[[r_sp_med-r_sp_down], [r_sp_up - r_sp_med]], color=color, capsize=5)
    #ax2.scatter(r_sp_med, -4*factor2, color=color, marker="o")
    #ax2.errorbar(0.23*(factor), der_med, yerr=[[der_med-der_down], [der_up-der_med]], color=color, capsize=5)
    #plt.scatter(0.23*(factor), der_med, color=color, marker="o")

        
    ax3 = ax2.twinx()
    kde = stats.gaussian_kde(r_sp)
    ax3.plot(r, kde(r), color=color, lw=2)
    #ax3.fill_between(r, 0, kde(r), color=color, alpha=0.1)
    idx = (r<r_sp_up) & (r>r_sp_down)
    ax3.fill_between(r[idx], 0, kde(r[idx]), color=color, alpha=0.3)
    ax3.plot([r_sp_up, r_sp_up], [0, kde(r_sp_up)], color=color, lw=2)
    ax3.plot([r_sp_down, r_sp_down], [0, kde(r_sp_down)], color=color, lw=2)
    ax3.set_ylim([0, 7])
    ax3.tick_params(axis='y', which='both', right=False, labelright=False)
    
    der_bin = np.linspace(0, -12, 500)
    ax3 = ax2.twiny()
    kde = stats.gaussian_kde(der)
    ax3.plot(kde(der_bin), der_bin, color=color, lw=2)
    #ax3.fill_betweenx(der_bin, 0, kde(der_bin), color=color, alpha=0.1)  
    idx = (der_bin < der_up) & (der_bin > der_down)
    ax3.fill_betweenx(der_bin[idx], 0, kde(der_bin[idx]), color=color, alpha=0.3)
    
    
    ax3.plot([0, kde(der_up)], [der_up, der_up], color=color, lw=2)
    ax3.plot([0, kde(der_down)], [der_down, der_down], color=color, lw=2)
    ax3.set_xlim([0, 4])
    ax3.tick_params(axis='x', which='both', top=False, labeltop=False)
    


# NFW fit
y_plotn2 = -1.*(1.+3.*r/0.547)/(1.+r/0.547)
ax2.plot(r, y_plotn2, color='k', label='NFW best-fit')

prof = NFW(Mvirarray[L.argmax()]*1e14*u.Msun, z_pivot, c=carray[L.argmax()])
model = prof.D_Sigma_reduced(CCCP.rbin.to('Mpc').value/(1.+z_pivot), CCCP.Sigma_crit.mean().to('Msun/Mpc/Mpc').value)
ax1.plot(CCCP.rbin.value, model/1e14, label='NFW best-fit', c='k')


#Hydrangea
y_sim = np.load('output/profiles/simulation_ds.npy')/1e14
r_sim = np.load('output/profiles/simulation_dsr.npy')
idx = (r_sim > 0.2) & (r_sim < 10)
ax1.plot(r_sim[idx], y_sim[idx], color='k', ls='--', label='Hydrangea')
#backend = emcee.backends.HDFBackend("output/chains/simulation.h5", read_only=True)
#data = backend.get_chain(flat = True, discard=n_disc)
#lnprob = backend.get_log_prob(flat=True, discard=n_disc)
#params_best = data[lnprob.argmax(), :]
#y_tot = rho(r, params_best)
#logy_tot = np.log(y_tot)
#derivative = np.gradient(logy_tot, logr)
y_g = np.load('output/profiles/simulation_g.npy')
r_sim = np.load('output/profiles/simulation_rg.npy')
ax2.plot(r_sim, y_g, color='k', ls='--', label='Hydrangea')


ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel("$R$ (Mpc)", size=20, multialignment='center')
ax1.set_ylabel("$\\langle g_t \\Sigma_\\mathrm{cr} \\rangle$ ($10^{14}$ M$_\\odot$ Mpc$^{-2}$)", multialignment='center')
ax1.tick_params(top=True, which='both', direction='inout')
ax1.tick_params(axis='y', which='minor')
ax1.tick_params(length=10, which='major')
ax1.tick_params(length=5, which='minor')
ax1.set_yticks([0.5,1, 2, 3])
ax1.yaxis.set_major_formatter(ScalarFormatter())
ax1.set_xticks([0.5,1, 2, 4, 8])
ax1.xaxis.set_major_formatter(ScalarFormatter())
ax1.set_xlim([0.2, 10])




ax2.set_xscale('log')
ax2.set_xlabel("$R$ (Mpc)", size=20, multialignment='center')
ax2.set_ylabel("$\\gamma= d \\log \\rho / d \\log r$", size=20, multialignment='center')



ax2.tick_params(top=True, which='both', direction='inout')
ax2.tick_params(length=10, which='major')
ax2.tick_params(length=5, which='minor')
ax2.set_xticks([0.5,1, 2, 4, 8])
ax2.xaxis.set_major_formatter(ScalarFormatter())
ax2.set_xlim([0.2, 10])
ax2.set_ylim([-6.5, 0.])

ax1.legend(loc='best', frameon=False)
ax2.legend(loc='best', frameon=False)
plt.tight_layout()
plt.savefig("output/figures/3.pdf")


***Show fit details from Fig 1***
=========

In [ ]:
import corner
plt.figure()
normL = L-L.max()
a = plt.scatter(Mvirarray[normL>-30], carray[normL>-30], c=normL[normL>-30])
plt.scatter(Mvirarray[normL.argmax()], carray[normL.argmax()], marker='x', s=100)
plt.colorbar(a)

plt.gca().set_title("Likelihood")
plt.gca().set_xlabel('M_vir (1e14 Msun)')
plt.gca().set_ylabel('c_vir')
plt.show()


samps = np.array([Mvirarray, carray]).T
corner.corner(samps, labels=['Mvir', 'c'], weights=np.exp(normL), show_titles=True, plot_datapoints=False, plot_density=False, quantiles=(0.16, 0.84), levels=(1-np.exp(-0.5),1-np.exp(-2)), fill_contours=True)#, truths=params_best)
plt.show()

***Figure 4***
========

In [ ]:
import corner
plt.rc('font', size=16)

n_disc = 5000
n_samples = 50000
r = np.linspace(.2, 100., 1000)
logr = np.log(r)



#LOAD FIT
filename = "output/chains/nomergers_True.h5"
backend = emcee.backends.HDFBackend(filename, read_only=True)
data = backend.get_chain(flat = True, discard=n_disc)


#Temp variables
params = data[np.random.choice(data.shape[0], n_samples), :]

r_sp = np.zeros(n_samples)
der = np.zeros(n_samples)
r_t = np.zeros(n_samples)
    
# Contours for model, derivative
for i in xrange(n_samples):
    
    #Total derivative 
    logy_tot = np.log(rho(r, params[i, :]))
    derivative = np.gradient(logy_tot, logr)

    r_sp[i] = r[derivative.argmin()]
    der[i] = derivative.min()
    r_t[i] = params[i, 3]

    
samps1 = np.copy(np.array([r_sp, der, r_t]).T)

labels=[r"$r_\mathrm{sp}$ $/$ Mpc", "$\gamma(r_\mathrm{sp})$", r"$r_t$ $/$ Mpc"]
figu = corner.corner(samps1, labels=labels, show_titles=False, plot_datapoints=False, plot_density=False, \
              levels=(1-np.exp(-0.5),1-np.exp(-2.)), fill_contours=True, title_kwargs={"fontsize": 14}, \
                smooth=0.8, hist_kwargs={'density': True, 'histtype':'stepfilled', 'label':'Gaussian prior on $r_t$'}, \
                color='red', bins=40, range=[0.999,0.999, 0.999], max_n_ticks=4)


#LOAD FIT
filename = "output/chains20/nomergers_True.h5"
backend = emcee.backends.HDFBackend(filename, read_only=True)
data = backend.get_chain(flat = True, discard=n_disc)


#Temp variables
params = data[np.random.choice(data.shape[0], n_samples), :]

r_sp = np.zeros(n_samples)
der = np.zeros(n_samples)
r_t = np.zeros(n_samples)

# Contours for model, derivative
for i in xrange(n_samples):
    
    #Total derivative 
    logy_tot = np.log(rho(r, params[i, :]))
    derivative = np.gradient(logy_tot, logr)

    r_sp[i] = r[derivative.argmin()]
    der[i] = derivative.min()
    r_t[i] = params[i, 3]


print "r_sp with flat prior:", np.percentile(r_sp, 50), np.percentile(r_sp, 100-15.86)-np.percentile(r_sp, 50), np.percentile(r_sp, 50)-np.percentile(r_sp, 15.86)
print "der with flat prior:", np.percentile(der, 50), np.percentile(der, 100-15.86)-np.percentile(der, 50), np.percentile(der, 50)-np.percentile(der, 15.86)



samps = np.array([r_sp, der, r_t]).T
labels=[r"$r_\mathrm{sp}$ $/$ Mpc", "$\gamma(r_\mathrm{sp})$", r"$r_t$ $/$ Mpc"]
corner.corner(samps, labels=labels, show_titles=False, plot_datapoints=False, plot_density=False, \
              levels=(1-np.exp(-0.5),1-np.exp(-2.)), fill_contours=False, title_kwargs={"fontsize": 14}, \
              fig=figu, hist_kwargs={'density': True, 'label':'Flat prior on $r_t$'}, \
              smooth=1.3, bins=40, no_fill_contours=True, range=[0.99,0.999, 0.999], max_n_ticks=4)


axes = np.array(figu.axes).reshape((3, 3))


#Histogram limits
axes[2, 2].set_ylim([0, 0.65])
axes[0, 0].set_ylim([0, 0.60])


#plot priors on rt
r_ts = np.linspace(0, 20, 1000)
axes[2, 2].plot(r_ts, 1./np.sqrt(2*np.pi)/2.*np.exp(-0.5*(r_ts-4.)**2./2./2.), ls='-', color='w')
axes[2, 2].plot(r_ts, 1./np.sqrt(2*np.pi)/2.*np.exp(-0.5*(r_ts-4.)**2./2./2.), ls=':', color='r')
axes[2, 2].axhline(1./20., ls=':', color='k')


plt.legend(frameon=False, loc='right', bbox_to_anchor=[0.6, 2.5])
#plt.tight_layout(h_pad=-0.3, w_pad=-0.3)
plt.savefig('output/figures/4.pdf')



***Figure 2***
========

In [ ]:
plt.figure(figsize=(8, 7))
e1psf = np.loadtxt("data/original/anisotropy.dat")[:, 0]
e1bias = np.loadtxt("data/original/anisotropy.dat")[:, 1]
d_e1bias = np.loadtxt("data/original/anisotropy.dat")[:, 2]

x = np.linspace(0, 0.15, 100)
y = x*0.04
plt.errorbar(e1psf, e1bias, yerr=d_e1bias, fmt='o', mfc='w', mec='k',ms=6, lw=2, c='k', capsize=3, label="H15")
#plt.plot(x, y, ls=':')

e1psf = np.loadtxt("data/original/anisotropy_new.dat")[:, 0]
e1bias = np.loadtxt("data/original/anisotropy_new.dat")[:, 1]
d_e1bias = np.loadtxt("data/original/anisotropy_new.dat")[:, 2]

plt.errorbar(e1psf, e1bias, yerr=d_e1bias, fmt='o', ms=6, lw=2, c='k',capsize=3, label="This work")

plt.axhline(0, ls='--', c='k')
plt.gca().set_xlabel('$e^{\mathrm{PSF}}_1$', size=20)
plt.gca().set_ylabel('$\\langle \\gamma_1 \\rangle$', size=20)
#plt.gca().set_ylim([0, 0.001])
plt.gca().set_xlim([0, 0.16])
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("output/figures/2.pdf")

In [ ]:
print "r_sp with flat prior:", np.percentile(r_sp, 50), "+", np.percentile(r_sp, 100-15.86)-np.percentile(r_sp, 50), "-", np.percentile(r_sp, 50)-np.percentile(r_sp, 15.86)
print "der with flat prior:", np.percentile(der, 50), "+", np.percentile(der, 100-15.86)-np.percentile(der, 50), "-", np.percentile(der, 50)-np.percentile(der, 15.86)

***APPENDIX***

In [ ]:
bin_edges = np.geomspace(.2, 9, 11)*u.Mpc

CCCP = cluster_sample('data/CCCPnomergers.fits', 'data/source/')
CCCP.stack_ESD(bin_edges, cosmic_noise='output/lss_covariance/', comoving=True, mscaling=False, weighted=True, r_limit=True)



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3.2))

lss_cov = CCCP.Cov*1.
stat_cov = np.zeros(CCCP.Cov.shape)*CCCP.Cov.unit
for i in xrange(len(CCCP.ESD_stat_err)):
    lss_cov[i, i] -=CCCP.ESD_stat_err[i]**2.
    stat_cov[i, i] = CCCP.ESD_stat_err[i]**2.

    
    
print np.log10(stat_cov.value).max()
print np.log10(lss_cov.value).max()
im = ax1.imshow(np.log10(stat_cov.value), vmin=22, vmax=28, cmap="Reds")
ax1.set_xticks([])
ax1.set_yticks([])
ax2.imshow(np.log10(lss_cov.value), cmap="Reds", vmin=22, vmax=28)
ax2.set_xticks([])
ax2.set_yticks([])



plt.tight_layout()
fig.subplots_adjust(right=0.7)
cbar_ax = fig.add_axes([0.75, 0.1, 0.02, 0.8])
bar = fig.colorbar(im, cax=cbar_ax)
bar.set_label('$\log_{10} \mathbf{C}^\mathrm{x}/(M_\odot \mathrm{Mpc}^2)^2$ ')
plt.savefig('output/figures/5.pdf')